In [4]:
import pandas as pd
import numpy as np
import time
import os
import json
from multiprocessing import Process, Manager, Value, Array, managers
from numpy import genfromtxt
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from multiprocessing import Pool
import collections
import itertools
from itertools import combinations
# from more_itertools import unique_everseen
from Bio.Seq import Seq
# from Bio import pairwise2

import os
import time
import pandas as pd
import numpy as np
from random import sample

import torch
import torch.nn as nn
import torch.nn.functional as F

from Bio import SeqIO
from multiprocessing import Process

import matplotlib.pyplot as plt

# cluster alignment and contribution entropy

## kalign

In [2]:
! /data/docker_qiime2_share_container_E/bioinformatics-tools/kalign/base_3.3.2/bin/kalign


Kalign (3.3.2)

Copyright (C) 2006,2019,2020,2021 Timo Lassmann

This program comes with ABSOLUTELY NO WARRANTY; for details type:
`kalign -showw'.
This is free software, and you are welcome to redistribute it
under certain conditions; consult the COPYING file for details.

Please cite:
  Lassmann, Timo.
  "Kalign 3: multiple sequence alignment of large data sets."
  Bioinformatics (2019) 
  https://doi.org/10.1093/bioinformatics/btz795


Usage: kalign  -i <seq file> -o <out aln> 

Options:

   --format           : Output format. [Fasta]
   --reformat         : Reformat existing alignment. [NA]
   --gpo              : Gap open penalty. [5.5]
   --gpe              : Gap extension penalty. [2.0]
   --tgpe             : Terminal gap extension penalty. [1.0]
   --version (-V/-v)  : Prints version. [NA]

Examples:

Passing sequences via stdin:

   cat input.fa | kalign -f fasta > out.afa

Combining multiple input files:

   kalign seqsA.fa seqsB.fa seqsC.fa -f fasta > combined.afa

[2023-0

! parallel -j 16 /data/docker_qiime2_share_container_E/bioinformatics-tools/kalign/base_3.3.2/bin/kalign \
-i /data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/E_coli_orthoslc/write_fasta/strict_core/{} \
-o /data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/E_coli_orthoslc/kalign/{} \
::: `ls /data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/E_coli_orthoslc/write_fasta/strict_core/` \
>  /data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/E_coli_orthoslc/log_kalign.txt 2>&1

## calculation

### Shannon entropy

$$ H = - \sum_{i=1}^{n} p_i \log_2 p_i $$

where $p_i$ is the probability of sequence $i$ in the gene cluster and $n$ is the total number of sequences in the gene cluster. The entropy quantifies the diversity of the sequences in the gene cluster. A high entropy value indicates high diversity, and a low entropy value indicates low diversity.

### my equition

$$C_{seqs} = \binom{list_{seqs}}{2}$$

$$S_{contribution} = \sum_{pair \in C_{seqs}} bi_zhi * get_distance(pairs[0], pairs[1]) * (a + b) * (1/1127)$$

$S_{contribution}$ Contrubution score of a gene cluster which reflect how much it affect tree inference.<br>
$list_{seqs}$ list of unique sequences in a gene cluster.<br>
$C_{seqs}$ combination of unique sequences pari in a gene cluster to be caculated.<br>
$C_{seqs}$ combination of unique sequences pari in a gene cluster to be caculated.<br>

In [3]:
list(combinations([1,2,3], 2))

[(1, 2), (1, 3), (2, 3)]

In [12]:
kaligned_path = 'E_coli_orthoslc/kalign/'

def get_distance(str1, str2):
    d = 0
    l = len(str1)
    
    for c in range(l):
        if str1[c] != str2[c]:
            d = d + 1
    d = d
    
    return d

def get_contribution(in_fasta_path, rt_dict):
    in_fasta = SeqIO.to_dict(SeqIO.parse(kaligned_path + in_fasta_path,
                                         'fasta')
                            )
    
    rep_d = {}
    
    for keys in in_fasta.keys():
        
        seq = str(in_fasta[keys].seq)
        
        if seq in rep_d.keys():
            rep_d[seq].append(keys)
        else:
            rep_d[seq] = [keys]
            
    key_lst = list(rep_d.keys())
    
    list_seqs = combinations(key_lst, 2)
    
    S_contribution = 0
    
    for pair in list_seqs:
#         a = len(pair[0])
#         b = len(pair[1])
#         if a < b:
#             bi_zhi = a/b
#         else:
#             bi_zhi = b/a
    
        a = len(rep_d[pair[0]]) # number of identical seqs of this representation in that cluster
        b = len(rep_d[pair[1]])
        
#         S_contribution = bi_zhi*get_distance(pair[0], pair[1])*(a + b)**1/len(in_fasta.keys()
#                                                                                        )
#                                                                                  )
        S_contribution = S_contribution + get_distance(pair[0], pair[1])*(a + b)
    
    rt_dict[in_fasta_path.replace('.fasta', '')] = [S_contribution, len(rep_d.keys()), len(pair[0])]
    
def divide_chunks(l, n):

    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n] 

```Python
kaligned_path = 'E_coli_orthoslc/kalign/'# !!!! need before alignment

def get_distance(str1, str2):
    d = 0
    l = len(str1)
    
    for c in range(len(str1)):
        if str1[c] != str2[c]:
            d = d + 1
    d = d
    
    return d

def get_contribution(in_fasta_path, rt_dict):
    in_fasta = SeqIO.to_dict(SeqIO.parse(kaligned_path + in_fasta_path,
                                         'fasta')
                            )
    
    rep_d = {}
    
    for keys in in_fasta.keys():
        
        seq = str(in_fasta[keys].seq)
        
        if seq in rep_d.keys():
            rep_d[seq].append(keys)
        else:
            rep_d[seq] = [keys]
            
    key_lst = list(rep_d.keys())
    
    list_seqs = combinations(key_lst, 2)
    
    S_contribution = 0
    
    for pair in list_seqs:
#         a = len(pair[0])
#         b = len(pair[1])
#         if a < b:
#             bi_zhi = a/b
#         else:
#             bi_zhi = b/a

        a = len(rep_d[pair[0]]) # number of identical seqs of this representation in that cluster
        b = len(rep_d[pair[1]])
        
#         S_contribution = S_contribution + bi_zhi*get_distance(pair[0], pair[1])*(a + b)*(1/1127)
        S_contribution = S_contribution + get_distance(pair[0], pair[1])*(a + b)*(1/1127)
    
    rt_dict[in_fasta_path.replace('.fasta', '')] = S_contribution
    
def divide_chunks(l, n):

    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]
```

In [13]:
start_time_external = time.time()
print(time.asctime())

if __name__ == '__main__':
    procc_num = 16
    
    mission_lst = list(divide_chunks(os.listdir(kaligned_path),
                                     procc_num
                                    )
                      )
    
    manager = Manager()
    return_dict = manager.dict()
    
    for submissions in mission_lst:
    
        jobs = []

        for x in range(0, len(submissions)):
            p = Process(target = get_contribution,
                        args = (submissions[x], 
                                return_dict
                               )
                       )
            p.start()
            jobs.append(p)


        for z in jobs:
            z.join()
        
end_time_external = time.time()
print("final time usage " + str(end_time_external - start_time_external))
print(time.asctime())

Wed Feb  8 22:38:43 2023
final time usage 129.2027509212494
Wed Feb  8 22:40:52 2023


In [14]:
df = pd.DataFrame.from_dict(return_dict, orient='index')
df = df.sort_values(by = [0], ascending = False)
df

,0,1,2
11002-12335,75319401,328,4358
10761-11925,27280561,238,2853
10416-10370,20438341,222,2818
10725-12709,19374959,202,2044
10938-12546,19092397,282,2769
...,...,...,...
11107-11787,4520,5,95
11057-11102,3389,4,87
11110-11410,3387,4,228
11109-13487,3381,2,85


In [15]:
df.to_csv('E_coli_orthoslc/cluster_contribution.csv',
          header = False)

# KO genome construction

In [5]:
df = pd.read_csv('E_coli_orthoslc/cluster_contribution.csv', 
                 header = None)

# with open('/data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/strains_id.json', 'r') as f_read:
#     read = json.load(f_read)
# print(len(read))

with open('E_coli_orthoslc/Step1_pre_res.txt', 'r') as f_read:
    rls = f_read.readlines()
    
    
d = {}
for x in rls:
    l_ = x.rstrip('\n').split('\t')
    d[l_[0]] = l_[1]
    
f_read.close()
    
# inv_map = {str(v): k for k, v in d.items()}
inv_map = d

In [6]:
df.iloc[0: 20, ]

,0,1,2,3
0,11002-12335,75319401,328,4358
1,10761-11925,27280561,238,2853
2,10416-10370,20438341,222,2818
3,10725-12709,19374959,202,2044
4,10938-12546,19092397,282,2769
5,10586-10463,18388556,199,1501
6,10671-12852,18176041,221,2390
7,10503-14515,17526699,220,2704
8,10961-13268,16291195,158,1140
9,10076-11950,15440429,185,1623


In [8]:
a = list(df[0])

# for x in a[715: ]:
for x in a[0: 50]:
    a.remove(x)

In [9]:
len(a)

675

to_skip = a[0: 1]
to_skip.extend(a[700: ])

a = []

In [10]:
to_skip = a
len(to_skip)

675

In [11]:
kaligned_path = '/data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/E_coli_orthoslc/kalign/'
final_core_genome_path = "/data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/E_coli_orthoslc/the_core_genome_1127.fasta"

to_write = []

seq_coll = {}

ini_seq = SeqIO.to_dict(SeqIO.parse(kaligned_path + "10004-14472.fasta", 
                                    "fasta")
                       )
for x in ini_seq:
    seq_coll[inv_map[x[0: 5]]] = ""

for x in os.listdir(kaligned_path):
#     print(x)
    x_fasta = SeqIO.to_dict(SeqIO.parse(kaligned_path + x, 
                                        "fasta")
                           )
    
    if x.replace('.fasta', '') in to_skip:
        for y in x_fasta:
            seq_coll[inv_map[y[0: 5]
                            ]
                    ] = seq_coll[inv_map[y[0: 5]
                                        ]
                                ] + '-' * len(x_fasta[y].seq
                                             )
    else:
        for y in x_fasta:
            seq_coll[inv_map[y[0: 5]
                            ]
                    ] = seq_coll[inv_map[y[0: 5]
                                        ]
                                ] + str(x_fasta[y].seq)

for z in seq_coll:
    to_write.append(SeqRecord(id = z, 
                              name = z,
                              description = "",
                              dbxrefs=[],
                              seq = Seq(seq_coll[z])
                             )
                   )
with open(final_core_genome_path, "w") as output_handle:
        SeqIO.write(to_write, output_handle, "fasta")

## for the phy

```Python
kaligned_path = '/data/docker_qiime2_share_G/Hejunwei/core_genome_study/genes_of_clusters/kaligned/'
final_core_genome_path = "/data/docker_qiime2_share_G/Hejunwei/core_genome_study/CNN/the_core_genome_1127_KO.phy"# ！！！！

to_write = []

seq_coll = {}

ini_seq = SeqIO.to_dict(SeqIO.parse(kaligned_path + "10001-14085.fasta", 
                                    "fasta")
                       )
for x in ini_seq:
    seq_coll[inv_map[x[0: 5]]] = ""

for x in os.listdir(kaligned_path):
#     print(x)
    x_fasta = SeqIO.to_dict(SeqIO.parse(kaligned_path + x, 
                                        "fasta")
                           )
    
    if x.replace('.fasta', '') in to_skip:
#         for y in x_fasta:
#             seq_coll[inv_map[y[0: 5]
#                             ]
#                     ] = seq_coll[inv_map[y[0: 5]
#                                         ]
#                                 ] + '-' * len(x_fasta[y].seq
#                                              )
        continue
    else:
        for y in x_fasta:
            seq_coll[inv_map[y[0: 5]
                            ]
                    ] = seq_coll[inv_map[y[0: 5]
                                        ]
                                ] + str(x_fasta[y].seq)

for z in seq_coll:
    to_write.append(SeqRecord(id = z, 
                              name = z,
                              description = "",
                              dbxrefs=[],
                              seq = Seq(seq_coll[z])
                             )
                   )
with open(final_core_genome_path, "w") as output_handle:
        SeqIO.write(to_write, output_handle, "phylip-relaxed")
````

koed_fasta = SeqIO.to_dict(SeqIO.parse(final_core_genome_path, 
                                    "phylip-relaxed")
                       )

len(koed_fasta['11368'].seq)

# ```Shell
#cd /data/docker_qiime2_share_G/Hejunwei/core_genome_study/CNN/RAxML_with_several_gene_left

cd /data/docker_qiime2_share_G/Hejunwei/core_genome_study/CNN/RAxML_with_top50_left
date ; raxmlHPC-PTHREADS-SSE3 \
-m GTRCAT -f a -p 1000 -# 1000 -x 1000 \
-s /data/docker_qiime2_share_G/Hejunwei/core_genome_study/CNN/the_core_genome_1127_KO.phy -n KOed_1127 -T 36 ; date
```

# pt np construction

In [12]:
# conv_d = {'A': [0],
#           'T': [1],
#           'C': [2],
#           'G': [3],
#           'Y': [1, 2], # T, C
#           'R': [0, 3], # A, G
#           'S': [2, 3], # C, G
#           'W': [0, 1], # A, T
#           'K': [1, 3], # G, Te
#           'M': [0, 2], # A, C
#           'B': [1, 2, 3], # T, C, G
#           'D': [0, 1, 3], # A, T, G
#           'H': [0, 1, 2], # A, T, C
#           'V': [0, 2, 3], # A, C, G
#           'N': [0, 1, 2, 3]
# #           '-': None
#          }

conv_d = {'A': [1., 0., 0., 0.],
          'T': [0., 1., 0., 0.],
          'C': [0., 0., 1., 0.],
          'G': [0., 0., 0., 1.],
          'Y': [0., 0.5, 0.5, 0.], # T, C
          'R': [0.5, 0., 0., 0.5], # A, G
          'S': [0., 0., 0.5, 0.5], # C, G
          'W': [0.5, 0.5, 0., 0.], # A, T
          'K': [0., 0.5, 0., 0.5], # G, T
          'M': [0.5, 0., 0.5, 0.], # A, C
          'B': [0., 1/3, 1/3, 1/3], # T, C, G
          'D': [1/3, 1/3, 0., 1/3], # A, T, G
          'H': [1/3, 1/3, 1/3, 0.], # A, T, C
          'V': [1/3, 0., 1/3, 1/3], # A, C, G
          'N': [0.25, 0.25, 0.25, 0.25],
          '-': [0., 0., 0., 0.]
         }

In [13]:
core_genome_fasta = SeqIO.to_dict(SeqIO.parse('/data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/E_coli_orthoslc/the_core_genome_1127.fasta',
                                              'fasta'
                                             )
                                 )

In [14]:
pt_path = '/data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/np/'

def one_hot_encoding(sequence, conv_d_):
    
    seq_tensor = [conv_d_[base] for base in sequence]
    seq_tensor = torch.tensor(seq_tensor).t()
    
    return seq_tensor

def DNA_to_pt(core_genomes, in_lst, convert_dict):
    
    for f in in_lst:
        s_seq = str(core_genomes[f].seq) 
        
        b_seq = one_hot_encoding(s_seq, convert_dict)
        
        torch.save(b_seq, 
                   pt_path
                   + f 
                   + '.pt')

len(core_genome_fasta['536'].seq)

position = 0
for x in core_genome_fasta['536'].seq:
    if x != '-':
        print(position)
        break
    else:
        position += 1

core_genome_fasta['536'].seq[0: 10]

test_tensor = torch.tensor(one_hot_encoding(str(core_genome_fasta['536'].seq[0: 10]), 
                                            conv_d)
                          ).t()

test_tensor

test_tensor
#test_tensor[:, :, 0: 10]

torch.save(test_tensor, 'np/test.pt')

loaded_sequence_tensor = torch.load('np/test.pt')

loaded_sequence_tensor

np_path = '/data/docker_qiime2_share_container_D/to_github/pt/CoreDL/pt/np/'

def DNA_to_np(core_genomes, in_lst, convert_dict):
    check_lst = list(convert_dict.keys())
    
    for f in in_lst:
        s_seq = str(core_genomes[f].seq) 
        
        b_seq = np.zeros([4, len(s_seq)], 
                         dtype = 'int'
                        )
        
        # vlauing
        for x in range(len(s_seq)):
            if s_seq[x] not in check_lst:
                continue
            else:
                length = len(convert_dict[s_seq[x]])
                
                for y in convert_dict[s_seq[x]]:
                    b_seq[convert_dict[s_seq[x]], x] = 1/length

        with open(np_path
                  + f 
                  + '.npy',
                  'wb') as f:
            np.save(f, b_seq)

In [15]:
start_time_external = time.time()
print(time.asctime())

if __name__ == '__main__':
    procc_num = 8
    
    mission_lst = np.array_split(list(core_genome_fasta.keys()), 
                                 procc_num
                                )
    
    jobs = []
    
    for x in range(0, procc_num):
        p = Process(target = DNA_to_pt,
                    args = (core_genome_fasta, 
                            mission_lst[x], 
                            conv_d
                           )
                   )
        p.start()
        jobs.append(p)
        
    
    for z in jobs:
        z.join()
        
end_time_external = time.time()
print("final time usage " + str(end_time_external - start_time_external))
print(time.asctime())

Wed Feb  8 23:16:56 2023
final time usage 43.908437728881836
Wed Feb  8 23:17:40 2023
